In [1]:
import mace
import glob
import os
import torch
import warnings
warnings.filterwarnings("ignore")
from mace.cli.run_train import main as mace_run_train_main
import sys
import logging

cuequivariance or cuequivariance_torch is not available. Cuequivariance acceleration will be disabled.


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
    print('Using GPU')

Using GPU


In [3]:
%%writefile mace-suite/runs/thymine/raw/config.yml
model: 'MACE'
foundation_model: 'mace-suite/foundation_model/MACE-OFF23_medium.model'
multiheads_finetuning: False
name: 'thymine_raw'
model_dir: 'mace-suite/runs/thymine/raw/models'
log_dir: 'mace-suite/runs/thymine/raw/logs'
results_dir: 'mace-suite/runs/thymine/raw/results'
device: cuda
heads:
  spin_flip_es1:
    train_file: 'mace-suite/data/processed_datasets/thymine/isolated/train_thymine_sf-es1.xyz'
    valid_file : 'mace-suite/data/processed_datasets/thymine/isolated/validation_thymine_sf-es1.xyz'
    energy_key: 'REF_energy'
    forces_key: 'REF_forces'
    E0s: 'isolated'
  spin_flip_es2:
    train_file: 'mace-suite/data/processed_datasets/thymine/isolated/train_thymine_sf-es2.xyz'
    valid_file : 'mace-suite/data/processed_datasets/thymine/isolated/validation_thymine_sf-es2.xyz'
    energy_key: 'REF_energy'
    forces_key: 'REF_forces'
    E0s: 'isolated'

max_num_epochs: 700
start_swa: 475
batch_size: 5
seed: 42


Overwriting mace-suite/runs/thymine/raw/config.yml


In [4]:
def train_mace(config_file_path):
    logging.getLogger().handlers.clear()
    sys.argv = ["program", "--config", config_file_path]
    mace_run_train_main()

In [5]:
train_mace('mace-suite/runs/thymine/raw/config.yml')

2025-08-12 09:03:31.566 INFO: ===========VERIFYING SETTINGS===========
2025-08-12 09:03:31.570 INFO: Stage Two is activated as start_stage_two was defined
2025-08-12 09:03:31.570 INFO: MACE version: 0.3.14
2025-08-12 09:03:31.577 INFO: CUDA version: 12.8, CUDA device: 0
2025-08-12 09:03:32.251 INFO: Using foundation model mace-suite/foundation_model/MACE-OFF23_medium.model as initial checkpoint.
2025-08-12 09:03:32.473 WARNING: Using multiheads finetuning with a foundation model that is not a Materials Project model, need to provied a path to a pretraining file with --pt_train_file.
2025-08-12 09:03:32.478 INFO: ===========LOADING INPUT DATA===========
2025-08-12 09:03:32.480 INFO: Using heads: ['spin_flip_es1', 'spin_flip_es2']
2025-08-12 09:03:32.481 INFO: Using the key specifications to parse data:
2025-08-12 09:03:32.481 INFO: spin_flip_es1: KeySpecification(info_keys={'energy': 'REF_energy', 'stress': 'REF_stress', 'virials': 'REF_virials', 'dipole': 'dipole', 'head': 'head', 'ele

In [ ]:
for file in glob.glob("mace-suite/runs/thymine/raw/models/*.pt"):
    os.remove(file)